In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def display_images(img_arr):
    """ Displays a horizontal row of images. """
    f, axarr = plt.subplots(nrows=1, ncols=len(img_arr), figsize=(10, 10))
    for i, img in enumerate(img_arr):
        plt.sca(axarr[i])
        plt.axis('off')
        plt.imshow(img_arr[i], cmap='gray')
    plt.show()

## Task 01

To implement dilation, we go over the original image section by section and assign 1 (or 255) to sections which contain at least 1 non-zero pixel.

In [ ]:
def dilation(image, SE_size):
  m, n = image.shape
  dilated_image = np.zeros((m,n))
  for i in range(m - SE_size):
    for j in range(n - SE_size):
      crop = image[i:i+SE_size, j:j+SE_size]
      if np.max(crop) != 0:
        dilated_image[i + (SE_size // 2), j + (SE_size // 2)] = 255
  
  return dilated_image

The following section shows the results of applying dilation using a 9x9 structuring element on a 100x100 image. Note how the white section is significantly thicker.

In [ ]:
image = cv2.imread('/content/FigP0905(U).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))
dilated_image = dilation(image, 9)
display_images([image, dilated_image])

Next, we look into applying dilation repeatedly. A structuring element of size 5x5 was used in this case. As we apply dilation more times, the white portion becomes increasingly thicker.

In [ ]:
images = []
image = cv2.imread('/content/FigP0905(U).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))
images.append(image)
dilated_image = dilation(image, 5)
images.append(dilated_image)
dilated_image = dilation(dilated_image, 5)
images.append(dilated_image)
dilated_image = dilation(dilated_image, 5)
images.append(dilated_image)
display_images(images)

Unlike dilation, erosion requires the entire structuing element to be contained insider the object to be eroded. From a code perspective, this means that the section we are working with must not contain any zeroes. If this is true, only then can the resulting image have a 1 (or 255) at the central pixel.

In [ ]:
def erosion(image, SE_size):
  m, n = image.shape
  eroded_image = np.zeros((m,n))
  for i in range(m - SE_size):
    for j in range(n - SE_size):
      crop = image[i:i+SE_size, j:j+SE_size]
      if np.all(crop):
        eroded_image[i + (SE_size // 2), j + (SE_size // 2)] = 255
  return eroded_image

Erosion on a 100x100 image using a 9x9 structuring element gives the following result. Notice how the white portion becomes thinner.

In [ ]:
image = cv2.imread('/content/FigP0905(U).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))
eroded_image = erosion(image, 9)
display_images([image, eroded_image])

Repeating the erosion process using a structuring element of size 5x5 shows that the white portion becomes increasingly thinner.

In [ ]:
images = []
image = cv2.imread('/content/FigP0905(U).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (100, 100))
images.append(image)
eroded_image = erosion(image, 5)
images.append(eroded_image)
eroded_image = erosion(eroded_image, 5)
images.append(eroded_image)
eroded_image = erosion(eroded_image, 5)
images.append(eroded_image)
display_images(images)

## Task 02

In [ ]:
image = cv2.imread('/content/Fig0941(a)(wood_dowels).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (300, 300))
plt.imshow(image, cmap='gray')
plt.show()

Before trying to find the sizes of the granules, it is required that we apply a smoothing operation. A morphological smoothing operation is thus performed, which invovles an opening operation followed by a closing  operation. A 5x5 square kernel was chosen for this.

In [ ]:
def smooth(image):
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
  image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
  image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
  return image

To determine the sizes of the granules in the image, we must apply the opening operation on the image repeatedly, with an increasing kernel size. An image of size 300x300 is being used.

Using a circular kernel to apply the opening operation will cause the granules to become smaller and intensity to decrease. At the point when the kernel size is exacltly 1px more than some granule's size, that granule will disappear. This disappearance will cause a noticeable drop in intensity, which we can observe using the following plot.

In [ ]:
def granulometry(image):
  image = smooth(image)
  intensities = []
  for i in range(1, 30):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (i, i))
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    intensities.append(np.average(opening))
  plt.plot(np.arange(len(intensities)), intensities, color ="red")

In [ ]:
image = cv2.imread('/content/Fig0941(a)(wood_dowels).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (300, 300))
granulometry(image)

Notice how there is a large drop at the x-axis values of 15 and 25. This means that the kernel sizes were 16 and 26 respectively at those points, which in turn means that the actual granules were of 15x15 pixels and 25x25 pixels respectively.

It should be noted that these values are dependent on the image size. Images resized to different dimensions will give different results.

## Task 03

This task requires us to answer the questions from exercise 9.34.

Textural segmentation allows us to find a boundary between structures of two distinct sizes in a single image. Since the image shown below seperates granules of different sizes into distinct sections, textural segmentation will work. The fact that the sections are circular will make no difference to the ability of textural segmentation to find the boundary.

In [ ]:
image = cv2.imread('/content/FigP0934(blobs_in_circular_arrangement).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (300, 300))
retVal, image = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
plt.imshow(image, cmap='gray')
plt.show()

The first step to performing textural segmentation is to perform a closing operation. The purpose is to make the entire section of smaller granules disappear, leaving a white background. This means we need to use circular kernels. The size of the kernels was found using trial and error, and a size of 25x25 was found to work well.

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
plt.imshow(image, cmap='gray')
plt.show()

The second step is to perform an opening operation with a kernel of a size equal to the largest gap between the remaining granules. This also invovled trial and error, but a kernel size of 50x50 was found to work well. Performing the opening operation with this kernel dilates the larger granules so that they fill the entire region, leaving a black area.

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
plt.imshow(image, cmap='gray')
plt.show()

Since we now have two distinct regions, one white and one black, we can find the boundary between them using a morphological gradient.

In [ ]:
kernel = np.ones((5,5),np.uint8)
gradient = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
plt.imshow(gradient, cmap='gray')
plt.show()

Finally, we can perform a binary AND operation with the inverted gradient and the original image to get an image with a black boundary dividing the granules of the two sizes.

The inverted gradient was used because the original gradient was a white line and the background of the image is also white.

In [ ]:
image = cv2.imread('/content/FigP0934(blobs_in_circular_arrangement).tif', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (300, 300))
retVal, image = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
image = cv2.bitwise_and(image, ~gradient)
plt.imshow(image, cmap='gray')
plt.show()

## Task 04

In [ ]:
image = cv2.imread('/content/FigP0936(bubbles_on_black_background).tif', cv2.IMREAD_GRAYSCALE)
retVal, image = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
plt.figure(figsize=(12, 6))
plt.imshow(image, cmap='gray')
plt.show()

To obtain the particles connected to the boundary, we can just take the connected component that starts at the (0, 0) index.

Unfortunately, the bottom border along with the particles connected to the bottom border is disconnected from the top border. As a workaround, the image is flipped, the component starting at (x, 0) is obtained and flipped again, and finally the union of the two outputs is taken.

In [ ]:
total_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)

for i in range(1, total_labels):
  x = stats[i, cv2.CC_STAT_LEFT]
  y = stats[i, cv2.CC_STAT_TOP]
  w = stats[i, cv2.CC_STAT_WIDTH]
  h = stats[i, cv2.CC_STAT_HEIGHT]
  area = stats[i, cv2.CC_STAT_AREA]

  if x == 0 and y == 0:
    top_border = (labels == i).astype("uint8") * 255
    
total_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(cv2.flip(image, 0), connectivity=4)
for i in range(1, total_labels):
  x = stats[i, cv2.CC_STAT_LEFT]
  y = stats[i, cv2.CC_STAT_TOP]
  w = stats[i, cv2.CC_STAT_WIDTH]
  h = stats[i, cv2.CC_STAT_HEIGHT]
  area = stats[i, cv2.CC_STAT_AREA]

  if y == 0:
    bottom_border = (labels == i).astype("uint8") * 255
    
boundary_image = cv2.bitwise_or(top_border, cv2.flip(bottom_border, 0))
plt.figure(figsize=(12, 6))
plt.imshow(boundary_image, cmap='gray')
plt.show()

If we read the original image again and remove the boundary particles, we will only be left with the overlapping and non-overlapping particles. If we retrieve all the connected components from these particles, we can then seperate out just the particles that are larger than a fixed size (400 px in area), the size of each of the non-overlapping particles. This size is unknown, so some trial and error was required to find it.

In [ ]:
image = cv2.imread('/content/FigP0936(bubbles_on_black_background).tif', cv2.IMREAD_GRAYSCALE)
retVal, image = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
image = image - boundary_image

total_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)

components = []
for i in range(1, total_labels):
  x = stats[i, cv2.CC_STAT_LEFT]
  y = stats[i, cv2.CC_STAT_TOP]
  w = stats[i, cv2.CC_STAT_WIDTH]
  h = stats[i, cv2.CC_STAT_HEIGHT]
  area = stats[i, cv2.CC_STAT_AREA]

  if area > 400:
    component = (labels == i).astype("uint8") * 255
    components.append(component)

overlap = cv2.bitwise_or(components[0], components[1], 0)
for i in range(2, len(components)):
  overlap = cv2.bitwise_or(components[i], overlap, 0)

plt.figure(figsize=(12, 6))
plt.imshow(overlap, cmap='gray')
plt.show()

Finally, we can remove both the bounday particles and the overlapping particles from the original image to obtain the non-overlapping particles.

In [ ]:
image = cv2.imread('/content/FigP0936(bubbles_on_black_background).tif', cv2.IMREAD_GRAYSCALE)
retVal, image = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
image = image - boundary_image
image = image - overlap
plt.figure(figsize=(12, 6))
plt.imshow(image, cmap='gray')
plt.show()